In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove6b300dtxt/glove.6B.300d.txt
/kaggle/input/potentialtalents/potential-talents - Aspiring human resources - seeking human resources.csv


# <font color='red'>Read the data</font> 

In [65]:
data = pd.read_csv('../input/potentialtalents/potential-talents - Aspiring human resources - seeking human resources.csv')

In [66]:
data.head(5)

,id,query Id,job_title,location,connection,fit
0,1,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,1,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,3,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,4,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [67]:
data['input'] = "Aspiring Human Resources seeking human resources"

### <font color='red'>Remove stop words and wordNetlemmatize</font>

In [68]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re 
import string 
def removestopwords(text):
    stop_words = set(stopwords.words('english'))
#     print(stop_words)
    text = re.sub(r'[^\w\s]', " ", text)
    text  = [word for word in text.split() if word not in stop_words]
    
    return text    

def lemmatization(text):
    words = removestopwords(text)
#     print(words)
#     words = word_tokenize(words)
#     word_tagged = pos_tag(words)
#     print( word_tagged)
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in words])
    return text

In [69]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [70]:
data['job_title'] = data['job_title'].apply(lambda x: lemmatization(x))
data['job_title'] 

0      2019 C T Bauer College Business Graduate Magna...
1      Native English Teacher EPIK English Program Korea
2                  Aspiring Human Resources Professional
3                    People Development Coordinator Ryan
4           Advisory Board Member Celal Bayar University
                             ...                        
99     Aspiring Human Resources Manager Graduating Ma...
100                   Human Resources Generalist Loparex
101            Business Intelligence Analytics Travelers
102                                   Always set Success
103        Director Of Administration Excellence Logging
Name: job_title, Length: 104, dtype: object

In [71]:
### <font color='red'>Remove stop words and wordNetlemmatize</font>

# <font color='red'>Word Embedding</font>

# **word2vec**

In [72]:
import nltk
import gensim
from gensim.models import Word2Vec
tokens = data['job_title'].apply(lambda x: nltk.word_tokenize(x))
w2v_model = Word2Vec(tokens,
                     min_count=1,
                     window=10,
                     vector_size=250,
                     alpha=0.03, 
                     min_alpha=0.0007,
                     workers = 4,
                     seed = 42)

In [73]:
from scipy import spatial
    
indextokey_set = set(w2v_model.wv.index_to_key)
def avg_feature_vector(sentence, w2v_model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, w2v_model.wv[word])
                
            if (n_words > 0):
                feature_vec = np.divide(feature_vec, n_words)
    return feature_vec  

In [74]:
score1 = []
for sentence in data['job_title']:
    s1_afv = avg_feature_vector(sentence, w2v_model, num_features=250, index2word_set=indextokey_set)     
    score1.append(s1_afv)

In [75]:
score2 = []
for sentence in data['input']:
    s2_afv = [avg_feature_vector(sentence, w2v_model, num_features=250, index2word_set=indextokey_set)]
    score2.append(s2_afv)

In [76]:

sim_word2vec = []
for i in range(len(score1)):
    sim = 1 - spatial.distance.cosine(score1[i],score2[i])
    sim_word2vec.append(sim)
print(sim_word2vec)

[-0.011893367394804955, 0.06679221242666245, 0.13390669226646423, 0.06176058575510979, 0.07493511587381363, 0.1933002769947052, -0.016195185482501984, 0.06752210855484009, -0.016195185482501984, -0.004719343036413193, 0.03147592768073082, -0.060703448951244354, -0.05785748362541199, -0.011893367394804955, -0.011893367394804955, 0.06679221242666245, 0.13390669226646423, 0.06176058575510979, -0.011893367394804955, 0.06679221242666245, 0.13390669226646423, 0.06176058575510979, 0.07493511587381363, 0.1933002769947052, -0.016195185482501984, 0.06752210855484009, 0.10875976830720901, 0.09487202018499374, 0.10875976830720901, 0.09487202018499374, -0.011893367394804955, 0.06679221242666245, 0.13390669226646423, 0.06176058575510979, 0.07493511587381363, 0.1933002769947052, -0.016195185482501984, 0.06752210855484009, -0.016195185482501984, -0.004719343036413193, 0.03147592768073082, -0.060703448951244354, -0.05785748362541199, -0.011893367394804955, 0.06679221242666245, 0.13390669226646423, 0.06

### <font color='red'>the output is Cosine simlilarity with word2vec embedding with input keywords</font>

# **tfidf**

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [78]:

sim_tfidf = []
for i in range(len(data['job_title'])):
    score = cosine_sim(data['job_title'][i], data['input'][i])
    sim_tfidf.append(score)
print(sim_tfidf)

[0.3311001718999398, 0.0, 0.6766282517941079, 0.0, 0.0, 0.6766282517941079, 0.45711076677231244, 0.0, 0.45711076677231244, 0.5051644862076965, 0.0, 0.0, 0.3677199804697471, 0.3311001718999398, 0.3311001718999398, 0.0, 0.6766282517941079, 0.0, 0.3311001718999398, 0.0, 0.6766282517941079, 0.0, 0.0, 0.6766282517941079, 0.45711076677231244, 0.0, 0.602232503337843, 0.6766282517941079, 0.602232503337843, 0.6766282517941079, 0.3311001718999398, 0.0, 0.6766282517941079, 0.0, 0.0, 0.6766282517941079, 0.45711076677231244, 0.0, 0.45711076677231244, 0.5051644862076965, 0.0, 0.0, 0.3677199804697471, 0.3311001718999398, 0.0, 0.6766282517941079, 0.0, 0.0, 0.6766282517941079, 0.45711076677231244, 0.0, 0.45711076677231244, 0.5051644862076965, 0.0, 0.0, 0.3677199804697471, 0.3311001718999398, 0.6766282517941079, 0.0, 0.6766282517941079, 0.0, 0.5051644862076965, 0.0, 0.0, 0.3677199804697471, 0.45711076677231244, 0.410995463934951, 0.4743307064971939, 0.335750593942534, 0.3480579450629847, 0.4109954639349

### <font color='red'>the output is Cosine simlilarity of tfidf vectorizer with input keywords</font>

# **Glove**

In [79]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

In [80]:
GLOVE_EMB = '../input/glove6b300dtxt/glove.6B.300d.txt'
EMBEDDING_DIM = 300

In [81]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['job_title'])

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)
word_index.keys()

Vocabulary Size : 192


dict_keys(['human', 'resources', 'aspiring', 'professional', 'student', 'seeking', 'college', 'generalist', 'university', 'specialist', 'business', 'english', 'coordinator', '2019', 'c', 't', 'bauer', 'graduate', 'magna', 'cum', 'laude', 'humber', 'management', 'manager', 'people', 'development', 'ryan', 'hr', 'senior', 'native', 'teacher', 'epik', 'program', 'korea', 'advisory', 'board', 'member', 'celal', 'bayar', 'hris', 'positions', 'chapman', 'svp', 'chro', 'marketing', 'communications', 'csr', 'officer', 'engie', 'houston', 'the', 'woodlands', 'energy', 'gphr', 'sphr', 'intercontinental', 'buckhead', 'atlanta', 'internship', 'opportunities', 'director', 'position', 'major', 'retail', 'staffing', 'north', 'america', 'information', 'systems', 'leader', 'experienced', 'recruiting', 'luxottica', 'groupe', 'beneteau', 'retired', 'army', 'national', 'guard', 'recruiter', 'office', 'scottmadden', 'inc', 'nortia', 'payroll', 'administrative', 'professionals', '408', '709', '2621', 'passi

In [82]:
embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [83]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [84]:
indextokey_set = set(word_index.keys())
print(indextokey_set)


{'create', 'schwan', 'experienced', 'success', 'service', 'intelligence', '2621', 'luxottica', 'endemol', 'programmer', 'member', 'atlanta', 'open', 'official', 'lead', 'retired', 'bayar', 'gis', 'arts', 'work', 'lab', 'international', 'staffing', 'executive', 'long', 'an', 'advisory', 'loparex', 'analyst', 'medical', 'focused', 'environmental', 'csr', 'professionals', 'engineer', 'travel', 'logging', 'styczynski', 'retail', 'intercontinental', 'bachelor', 'japan', 'energy', 'university', 'aspiring', 'communications', 'leader', 'employment', 'ey', 'chapman', 'st', 'coordinator', 'opportunity', 'travelers', 'recruiting', 'compensation', 'organization', 'may', 'resources', 'level', 'data', 'management', 'officer', 'gphr', 'love', 'program', '2019', 'celal', 'professional', 'national', 'louis', 'state', 'benefits', 'information', 'position', 'development', 'major', 'mes', 'illinois', 'student', 'graduating', 'magna', 'patient', 'conflict', 'c', 'sphr', 'hr', 'internship', 'entry', '408', 

In [85]:
def avg_feature_vector_glove(sentence,embedding_matrix, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec,embedding_matrix[word_index[word]]) 
            if (n_words > 0):
                feature_vec = np.divide(feature_vec, n_words)
    return feature_vec  

In [86]:
score_1 = []
for sentence in data['job_title']:
    s1_afv = avg_feature_vector_glove(sentence,embedding_matrix,num_features=300, index2word_set=indextokey_set)     
    score_1.append(s1_afv)
print(len(score_1))
    

104


In [87]:
score_2 = []
for sentence in data['input']:
    s1_afv = avg_feature_vector_glove(sentence,embedding_matrix, num_features=300, index2word_set=indextokey_set)     
    score_2.append(s1_afv)


In [88]:

sim_glove = []
for i in range(len(score1)):
    sim = 1 - spatial.distance.cosine(score_1[i],score_2[i])
    sim_glove.append(sim)
print(sim_glove)

[0.2509136398602295, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.2509136398602295, 0.2509136398602295, 1, 1, 1, 0.2509136398602295, 1, 1, 1, 1, 1, 1, 1, 0.2942120022683197, 1, 0.2942120022683197, 1, 0.2509136398602295, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.2509136398602295, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.2509136398602295, 1, 1, 1, 1, 1, 1, 1, 1, 0.10024384905543438, 1, 1, 1, 0.4731049562565006, 1, 1, 0.3640385907915118, 1, 0.019371460168884136, 0.5785132406495018, 1, 1, 1, 1, 1, 0.1461711292934662, 1, 0.3490409728677196, 1, 0.5764968445625774, 1, 1, 1, 1, 1, 0.5446279634976414, 0.23027859878715806, 0.29523922556225135, 1, 1, 1, 1, 1, 0.18970171815088643, 1, 1, 0.3306740991118384, 1]


/opt/conda/lib/python3.7/site-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


### <font color='red'>the output is Cosine simlilarity glove word embedding with input keywords </font>

# **Cosine Similarity**

In [89]:
# Cosine Similarity with word2vec, tfidf, and glove embedding 
result_similarity = []
for i in range(len(sim_word2vec)):
    score_mean = (sim_word2vec[i]+sim_tfidf[i]+sim_glove[i])/3
    result_similarity.append(score_mean)
print(result_similarity)

[0.19004014812178813, 0.3555974041422208, 0.6035116480201906, 0.35392019525170326, 0.3583117052912712, 0.623309509596271, 0.48030519376327013, 0.35584070285161334, 0.48030519376327013, 0.5001483810570945, 0.3438253092269103, 0.31309885034958523, 0.43662083228144505, 0.19004014812178813, 0.19004014812178813, 0.3555974041422208, 0.6035116480201906, 0.35392019525170326, 0.19004014812178813, 0.3555974041422208, 0.6035116480201906, 0.35392019525170326, 0.3583117052912712, 0.623309509596271, 0.48030519376327013, 0.35584070285161334, 0.3350680913044572, 0.5905000906597005, 0.3350680913044572, 0.5905000906597005, 0.19004014812178813, 0.3555974041422208, 0.6035116480201906, 0.35392019525170326, 0.3583117052912712, 0.623309509596271, 0.48030519376327013, 0.35584070285161334, 0.48030519376327013, 0.5001483810570945, 0.3438253092269103, 0.31309885034958523, 0.43662083228144505, 0.19004014812178813, 0.3555974041422208, 0.6035116480201906, 0.35392019525170326, 0.3583117052912712, 0.623309509596271, 

### <font color='red'>the output is sum of Cosine Similarity of word2vec embedding, tfidf embedding and glove embedding with input keywords</font>

In [90]:
data['result_similarity'] = result_similarity

In [91]:
data['result_similarity']

0      0.190040
1      0.355597
2      0.603512
3      0.353920
4      0.358312
         ...   
99     0.227842
100    0.525331
101    0.330686
102    0.099878
103    0.324775
Name: result_similarity, Length: 104, dtype: float64

In [92]:
# cut the connection variable as 1,2,3,4, the larger the higher rank 
data['connection'] = data['connection'].str.split('+').str[0]
data['connection'] = data['connection'].astype(int)
list = data['connection'].tolist()
list.sort()
# print(list)
data['connection'] = pd.cut(data['connection'],[0,40,200,499,600],labels=[1,2,3,4]) 
data['connection'] = data['connection'].astype('int')

In [93]:

dum_key = pd.get_dummies(data['location'])
data = data.drop('location', 1)
data = pd.concat([data,dum_key],axis=1)
data.head(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,id,query Id,job_title,connection,fit,input,result_similarity,Amerika Birleşik Devletleri,"Atlanta, Georgia","Austin, Texas Area",...,"Milpitas, California","Monroe, Louisiana Area","Myrtle Beach, South Carolina Area","New York, New York","Raleigh-Durham, North Carolina Area",San Francisco Bay Area,"San Jose, California","Torrance, California","Virginia Beach, Virginia","İzmir, Türkiye"
0,1,1,2019 C T Bauer College Business Graduate Magna...,2,NaN,Aspiring Human Resources seeking human resources,0.190040,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,Native English Teacher EPIK English Program Korea,4,NaN,Aspiring Human Resources seeking human resources,0.355597,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,Aspiring Human Resources Professional,2,NaN,Aspiring Human Resources seeking human resources,0.603512,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,3,People Development Coordinator Ryan,4,NaN,Aspiring Human Resources seeking human resources,0.353920,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4,Advisory Board Member Celal Bayar University,4,NaN,Aspiring Human Resources seeking human resources,0.358312,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### <font color='red'>the output is one hot encoding location variable</font>

In [94]:

data['result_similarity_bin'] = pd.cut(data['result_similarity'],[0,0.12,0.18,0.24,0.3,0.36,0.42,0.48,0.54,0.6,0.66],labels=[1,2,3,4,5,6,7,8,9,10])
data['result_similarity_bin']

0       3
1       5
2      10
3       5
4       5
       ..
99      3
100     8
101     5
102     1
103     5
Name: result_similarity_bin, Length: 104, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

### <font color='red'>the output is to cut the result_similarity variable as 1-10, the larger the higher rank </font>

In [95]:
data.columns

Index(['id', 'query Id', 'job_title', 'connection', 'fit', 'input',
       'result_similarity', 'Amerika Birleşik Devletleri', 'Atlanta, Georgia',
       'Austin, Texas Area', 'Baltimore, Maryland',
       'Baton Rouge, Louisiana Area', 'Bridgewater, Massachusetts',
       'Cape Girardeau, Missouri', 'Chattanooga, Tennessee Area',
       'Chicago, Illinois', 'Dallas/Fort Worth Area', 'Denton, Texas',
       'Gaithersburg, Maryland', 'Greater Atlanta Area', 'Greater Boston Area',
       'Greater Chicago Area', 'Greater Grand Rapids, Michigan Area',
       'Greater Los Angeles Area', 'Greater New York City Area',
       'Greater Philadelphia Area', 'Highland, California', 'Houston, Texas',
       'Houston, Texas Area', 'Jackson, Mississippi Area', 'Kanada',
       'Katy, Texas', 'Kokomo, Indiana Area', 'Lafayette, Indiana',
       'Lake Forest, California', 'Las Vegas, Nevada Area',
       'Long Beach, California', 'Los Angeles, California',
       'Milpitas, California', 'Monroe, Louisi

In [96]:

def Starcandidate(data,col1,col2,text):
    #select the candidate job title which match the engine query 
    data_star = data.loc[data[col1].str.contains(text)]
    data_star[col2] =1
#     print(data_star[col2])
    data_remining = data.loc[~data[col1].str.contains(text)]
#     print( data_remining)
    for value in data_remining[col2].values:
        if value >=0.5:
             data_remining[col2] = 1
        else:  data_remining[col2] =0 
    data = pd.concat([data_star,data_remining])
    return data
         
    
        
    
        

## <font color='red'>This function is writen for assign the result_similarity column 0 or 1 </font>

In [97]:
data = Starcandidate(data,'job_title','result_similarity',"Aspiring Human Resources|Seeking Human Resources")
data.sample(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,id,query Id,job_title,connection,fit,input,result_similarity,Amerika Birleşik Devletleri,"Atlanta, Georgia","Austin, Texas Area",...,"Monroe, Louisiana Area","Myrtle Beach, South Carolina Area","New York, New York","Raleigh-Durham, North Carolina Area",San Francisco Bay Area,"San Jose, California","Torrance, California","Virginia Beach, Virginia","İzmir, Türkiye",result_similarity_bin
92,93,17,Admissions Representative Community medical ce...,1,NaN,Aspiring Human Resources seeking human resources,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
91,92,16,Seeking employment opportunity within Customer...,2,NaN,Aspiring Human Resources seeking human resources,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
29,30,1,Seeking Human Resources Opportunities,3,NaN,Aspiring Human Resources seeking human resources,1,0,0,0,...,0,0,0,0,0,0,0,0,0,9
66,67,9,Human Resources Staffing Recruiting Professional,4,NaN,Aspiring Human Resources seeking human resources,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
24,25,1,Student Humber College Aspiring Human Resource...,2,NaN,Aspiring Human Resources seeking human resources,1,0,0,0,...,0,0,0,0,0,0,0,0,0,8
63,64,7,SVP CHRO Marketing Communications CSR Officer ...,4,NaN,Aspiring Human Resources seeking human resources,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
21,22,3,People Development Coordinator Ryan,4,NaN,Aspiring Human Resources seeking human resources,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
23,24,1,Aspiring Human Resources Specialist,1,NaN,Aspiring Human Resources seeking human resources,1,0,0,0,...,0,0,0,0,0,0,0,0,0,10
87,88,9,Human Resources Management Major,1,NaN,Aspiring Human Resources seeking human resources,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
26,27,1,Aspiring Human Resources Management student se...,4,NaN,Aspiring Human Resources seeking human resources,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [98]:

y = data['result_similarity']
x = data.drop(['id','fit','job_title','input','result_similarity'], axis=1)

## <font color='red'>set result_similarity column as label</font>

In [99]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split( x, y,test_size=0.7, random_state=26)

In [100]:
train_baskets = x_train.groupby(['query Id'])['result_similarity_bin'].count().values

In [101]:
train_baskets 

array([16,  3,  1,  1,  1,  1,  5,  1,  1,  1])

In [102]:
!pip install lightgbm==2.2.3 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## <font color='red'>Apply Learning TO Rank Algorithmn</font>

In [103]:
from lightgbm.sklearn import LGBMRanker
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [104]:
ranker = ranker.fit(
    x_train,
    y_train,
    group=train_baskets,
)

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 7
[LightGBM] [Info] Number of data: 31, number of used features: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements


In [105]:
x_test['preds'] = ranker.predict(x_test)


In [106]:
# predict the rank based on query Id and prediction
pred_dict = x_test \
    .sort_values(['query Id', 'preds'], ascending=False) \
    .groupby('query Id')['result_similarity_bin']

In [107]:
for rank, group in pred_dict:
    print('name is {}'.format(name))
    print('================================================')
    print('rank is {}'.format(group))


name is 21
rank is 93     4
24     8
35    10
98    10
16    10
45    10
99     3
49     8
9      8
71     8
8      8
61     8
6      8
72     8
23    10
13     3
65     3
28     5
0      3
5     10
26     5
32    10
59    10
74     1
27     9
52     8
2     10
14     3
75     5
Name: result_similarity_bin, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
name is 21
rank is 1     5
44    5
Name: result_similarity_bin, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
name is 21
rank is 58    5
21    5
33    5
3     5
17    5
Name: result_similarity_bin, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
name is 21
rank is 22    5
47    5
4     5
Name: result_similarity_bin, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
name is 21
rank is 50    5
60    5
7     5
25    5
Name: result_similarity_bin, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
name is 21
r

## <font color='red'>The output shows the rank of group of similatrity with input keywords</font>